In [1]:
from datasets import load_dataset

In [9]:
# fw = load_dataset("HuggingFaceFW/fineweb-edu", name="sample-10BT", split="train")

ds = load_dataset("airtrain-ai/fineweb-edu-fortified", data_dir="data")

Resolving data files:   0%|          | 0/817 [00:00<?, ?it/s]

train-00000-of-00013.parquet:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

train-00001-of-00013.parquet:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

In [12]:
ds["train"]

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score', 'embedding', 'count'],
    num_rows: 10800000
})

In [6]:
filtered_ds = ds.filter(lambda x: x["score"]>=3)

Filter:   0%|          | 0/10800000 [00:00<?, ? examples/s]

In [11]:
train_test = filtered_ds["train"].train_test_split(test_size=.2, seed=69)
train_val = train_test["train"].train_test_split(test_size=.1, seed=69)

train_ = train_val["train"]
val_ = train_val["test"]
test_ = train_test["test"]

train_.save_to_disk("data/train_data", num_proc=8)
val_.save_to_disk("data/val_data", num_proc=8)
test_.save_to_disk("data/test_data", num_proc=8)

Saving the dataset (0/45 shards):   0%|          | 0/3360181 [00:00<?, ? examples/s]

Saving the dataset (0/8 shards):   0%|          | 0/373354 [00:00<?, ? examples/s]

Saving the dataset (0/13 shards):   0%|          | 0/933384 [00:00<?, ? examples/s]

In [1]:
import multiprocessing

In [2]:
multiprocessing.cpu_count()

16